In [ ]:
import json
import uuid

import boto3
from boto3.dynamodb.conditions import Key

dyn_resource = boto3.resource("dynamodb")

TABLE_NAME = "ipynb-test-table"

In [ ]:
# Delete table if it already exists
try:
    table = dyn_resource.Table(TABLE_NAME)
    table.delete()
    table.wait_until_not_exists()
    print(f"Deleted table {TABLE_NAME}")
except Exception as e:
    print("Table didn't exist yet, you're good.")

In [ ]:
def create_ddb_table(table_name=TABLE_NAME):
    params = {
        "TableName": table_name,
        "KeySchema": [
            {"AttributeName": "partition_key", "KeyType": "HASH"},
            # {"AttributeName": "sort_key", "KeyType": "RANGE"},
        ],
        "AttributeDefinitions": [
            {"AttributeName": "partition_key", "AttributeType": "S"},  # S is string
            # {"AttributeName": "sort_key", "AttributeType": "S"},
        ],
        "ProvisionedThroughput": {"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    }
    table = dyn_resource.create_table(**params)
    print(f"Creating {table_name}...")
    table.wait_until_exists()
    print("Done.")
    return

In [ ]:
create_ddb_table()

In [ ]:
# Grab a pointer to the table
table = dyn_resource.Table(name=TABLE_NAME)

In [ ]:
# Write some data to table
# Use random UUID as partition key
# Create a field called "some_data" with a string in it
this_uuid = str(uuid.uuid4())
this_data = "This is some david data!"
response = table.put_item(
    Item={
        "partition_key": this_uuid,
        "some_data": this_data,
        "username": "david"
    }
)

this_uuid = str(uuid.uuid4())
this_data = "This is some kazu data!"
response = table.put_item(
    Item={
        "partition_key": this_uuid,
        "some_data": this_data,
        "username": "kazu"
    }
)

In [ ]:
# Get the data back from the table based on partition_key
kazu: dict = table.get_item(Key={"partition_key": this_uuid})
print(kazu["Item"]["some_data"])

In [ ]:
# Test query functionality
# Query on primary key (you can additionally query on sort key)
key_condition_expression = Key("partition_key").eq(this_uuid)
qkazu: dict = table.query(KeyConditionExpression=key_condition_expression)
print(f"Found {len(qkazu['Items'])} values from query.")
print(qkazu["Items"][0])

In [ ]:
# Scan table with key
scan_kwargs = {
        "FilterExpression": Key("username").eq("david"),
        "ProjectionExpression": "some_data, #foo",
        "ExpressionAttributeNames": {"#foo": "partition_key"},
    }
scan_results = table.scan(**scan_kwargs)
print(scan_results["Items"])

In [ ]:
# Let's make sure we can store json data (by dump to string)
j_uuid = str(uuid.uuid4())
j_data = {"foo": {"bar": 4}}
table.put_item(
    Item={
        "partition_key": j_uuid,
        "some_data": json.dumps(j_data),
    }
)

In [ ]:
# Get the json data back from the table based on partition_key
kazu: dict = table.get_item(Key={"partition_key": j_uuid})
retrieved_j_data = json.loads(kazu["Item"]["some_data"])
print(retrieved_j_data)

In [ ]:
# How do I add another row to existing partition?
def update_entry(primary_key, new_col_name, new_col_value):
    # Define the key of the item you want to update
    key = {"partition_key": primary_key}

    # Define the attribute(s) you want to add or update
    update_expression = "SET #new_attr = :new_value"
    expression_attribute_names = {"#new_attr": new_col_name}
    expression_attribute_values = {":new_value": new_col_value}

    # Update the item
    table.update_item(
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeNames=expression_attribute_names,
        ExpressionAttributeValues=expression_attribute_values,
    )

In [ ]:
update_entry(this_uuid, "NEW DATA!", "This is some new data!")

In [ ]:
kazu: dict = table.get_item(Key={"partition_key": this_uuid})
print(kazu["Item"])

In [ ]:
# And overwriting existing columns?
update_entry(this_uuid, "NEW DATA!", "Does this overwrite?")

In [ ]:
kazu: dict = table.get_item(Key={"partition_key": this_uuid})
print(kazu["Item"])

In [ ]:
# For my use case, the first lambda will create a unique id
# that is the partition key. It will create a dynamodb entry
# with the parent media URI as data.
# Future lambdas will all only know about that partition key.

In [ ]:
prod_table = dyn_resource.Table(name="MAS-App-Table")

# Scan and return everything in the table
scan_kwargs = {
    # "FilterExpression": Key("year").between(year_range["first"], year_range["second"]),
    "ProjectionExpression": "media_uri, job_creation_time, #foo",
    "ExpressionAttributeNames": {"#foo": "UUID"},
}
blah = prod_table.scan(**scan_kwargs)

blah["Items"][0].keys()